In [ ]:
%cd ..

In [ ]:
from damply import dirs
import SimpleITK as sitk
from readii.image_processing import displayCTSegOverlay

In [ ]:
image_path = dirs.PROCDATA / "TCIA_NSCLC-Radiomics/images/mit_all_GTVs_NSCLC_Radiomics/LUNG1-294_0293/CT_74742304/CT.nii.gz"
mask_path = dirs.PROCDATA / "TCIA_NSCLC-Radiomics/images/mit_all_GTVs_NSCLC_Radiomics/LUNG1-294_0293/RTSTRUCT_02337966/GTV__[GTV-1].nii.gz"

In [ ]:
image = sitk.ReadImage(image_path)
mask = sitk.ReadImage(mask_path)

In [ ]:
displayCTSegOverlay(image, mask)

In [ ]:
displayCTSegOverlay(image, mask, crop=True)

# 3D Visualization

In [2]:
import pyvista as pv
from pyvista import examples
from pathlib import Path
import numpy as np

In [ ]:
mesh = examples.download_knee_full()

pl = pv.Plotter()
pl.add_volume(mesh, cmap="bone", opacity="sigmoid", show_scalar_bar=False)
pl.enable_depth_peeling()
pl.show()

In [ ]:
filename = Path('../data/procdata/TCIA_NSCLC-Radiomics/images/mit_all_GTVs_NSCLC_Radiomics/LUNG1-294_0293/RTSTRUCT_02337966/GTV__[GTV-1].nii.gz')
reader = pv.get_reader(filename)
mask = reader.read()

clim = [1]
pl = pv.Plotter()
pl.add_volume(mask, cmap='cool', opacity="sigmoid", show_scalar_bar=False)
pl.enable_depth_peeling()
pl.show()


## Example heart segmentation from PyVista

https://docs.pyvista.org/examples/02-plot/volume_rendering.html#volume-with-segmentation-mask

In [ ]:
dataset = examples.download_whole_body_ct_female()
ct_image = dataset['ct']
heart_mask = dataset['segmentations']['heart']

In [ ]:
heart_array = np.full_like(ct_image.active_scalars, -1000)

ct_image_array = ct_image.active_scalars
heart_mask_array = heart_mask.active_scalars

heart_array[heart_mask_array == True] = ct_image_array[heart_mask_array == True]

ct_image['heart'] = heart_array

In [ ]:
pl = pv.Plotter()

# Add the CT image.
pl.add_volume(
    ct_image,
    scalars='NIFTI',
    cmap='bone',
    opacity='sigmoid_15',
    show_scalar_bar=False,
)

# Add masked CT image of the heart and use a contrasting color map.
_ = pl.add_volume(
    ct_image,
    scalars='heart',
    cmap='gist_heat',
    opacity='linear',
    opacity_unit_distance=np.mean(ct_image.spacing),
)

# Orient the camera to provide a latero-anterior view.
pl.view_yz()
pl.camera.azimuth = 70
pl.camera.up = (0, 0, 1)
pl.camera.zoom(1.5)
pl.show()

## Example Visualize Anatomical Groups

In [ ]:
from __future__ import annotations

import pyvista as pv
from pyvista import examples

In [ ]:
dataset = examples.download_whole_body_ct_female()

label_map = dataset['label_map']

# Get a list of all label names. This list will be filtered by group.
label_names = dataset['segmentations'].keys()

# Get color and id mappings included with the dataset. These are used to filter and
# color the contours.
names_to_colors = dataset.user_dict['names_to_colors']
names_to_ids = dataset.user_dict['names_to_ids']
ids_to_colors = dataset.user_dict['ids_to_colors']

In [ ]:
def filter_labels(label_names: list[str], search_terms: list[str]):
    def include_label(label_name: str):
        return any(target in label_name for target in search_terms)

    return [label for label in label_names if include_label(label)]


def plot_anatomy(search_terms: list[str]):
    # Get a list of labels which contain any of the listed terms.
    group_names = filter_labels(label_names, search_terms)

    # Get the label ids corresponding to the matched labels.
    group_ids = [names_to_ids[name] for name in group_names]

    # Selectively generate surfaces for the specified labels
    group_surface = dataset['label_map'].contour_labels(select_inputs=group_ids)
    print(group_surface)

    # Color the labels with the color mapping
    colored_surface = group_surface.color_labels(colors=ids_to_colors)
    print(colored_surface)

    # # Plot the label map.
    # pl = pv.Plotter()
    # pl.add_mesh(colored_surface)
    # pl.view_zx()
    # pl.camera.up = (0, 0, 1)
    # pl.show()

In [ ]:
# Define terms which describe all relevant segments.
cardio = [
    'heart',
    # 'aorta',
    # 'artery',
    # 'brachiocephalic_trunk',
    # 'vein',
    # 'atrial_appendage',
    # 'vena_cava',
]

# Plot the labels associated with these terms.
plot_anatomy(cardio)

## NSCLC-Radiomics sample

In [10]:
image_path = Path("../data/procdata/TCIA_NSCLC-Radiomics/images/mit_all_GTVs_NSCLC_Radiomics/LUNG1-294_0293/CT_74742304/CT.nii.gz")

mask_paths = [path for path in Path('../data/procdata/TCIA_NSCLC-Radiomics/images/mit_all_GTVs_NSCLC_Radiomics/LUNG1-294_0293').rglob('GTV__*.nii.gz')]

In [13]:
image = pv.read(image_path)
# remove bottom layer of image with table
image = image.crop(margin=(0,125,0))
image_array = image.active_scalars

masks = {str(mask_path.name).removesuffix('.nii.gz'): pv.read(mask_path).crop(margin=(0,125,0)) for mask_path in mask_paths}

In [14]:
for gtv_name, gtv_mask in masks.items():
    gtv_array = np.full_like(image_array, 0)

    gtv_mask_array = gtv_mask.active_scalars

    gtv_array[gtv_mask_array == True] = 1

    image[gtv_name] = gtv_array

    _ = pl.add_volume(
        image,
        scalars = gtv_name,
        cmap='gist_heat',
        opacity='linear',
        opacity_unit_distance=np.mean(image.spacing)
    )


In [15]:
image

ImageData (0x30efb9060)
  N Cells:      11870019
  N Points:     12072960
  X Bounds:     0.000e+00, 4.990e+02
  Y Bounds:     1.221e+02, 3.770e+02
  Z Bounds:     0.000e+00, 2.670e+02
  Dimensions:   512, 262, 90
  Spacing:      9.766e-01, 9.766e-01, 3.000e+00
  N Arrays:     6

In [ ]:
pl = pv.Plotter()
pl.enable_depth_peeling(10)

# Add the base image
pl.add_volume(
    image,
    scalars='NIFTI',
    cmap='gray',
    opacity='sigmoid_9',
    show_scalar_bar=False,  
)

_ = pl.add_volume(
        image,
        scalars = "GTV__[GTV-1]",
        cmap=['white', 'cyan'],
        opacity='linear',
        opacity_unit_distance=np.mean(image.spacing),
        show_scalar_bar=False,
        shade = True
    )

_ = pl.add_volume(
        image,
        scalars = "GTV__[gtv-2]",
        cmap=['white', 'magenta'],
        opacity='linear',
        # opacity_unit_distance=np.mean(image.spacing),
        show_scalar_bar=False,
        shade = True
    )

_ = pl.add_volume(
        image,
        scalars = "GTV__[gtv-5]",
        cmap=['white', 'red'],
        opacity='linear',
        opacity_unit_distance=np.mean(image.spacing),
        show_scalar_bar=False,
        shade = True
    )

_ = pl.add_volume(
        image,
        scalars = "GTV__[gtv-6]",
        cmap=['white', 'lime'],
        opacity='linear',
        opacity_unit_distance=np.mean(image.spacing),
        show_scalar_bar=False,
        shade = True
    )

_ = pl.add_volume(
        image,
        scalars = "GTV__[gtv-7]",
        cmap=['white', 'yellow'],
        opacity='linear',
        opacity_unit_distance=np.mean(image.spacing),
        show_scalar_bar=False,
        shade = True
    )


pl.view_xz()

pl.camera.zoom(1.5)
# pl.camera.position = (0, 0, 0)
pl.camera.azimuth = 0
pl.show()


Widget(value='<iframe src="http://localhost:50491/index.html?ui=P_0x4b69f1b80_23&reconnect=auto" class="pyvist…

## GIF creation

In [73]:
pl = pv.Plotter(notebook=False, off_screen=True)
pl.enable_depth_peeling(10)

# Add the base image
pl.add_volume(
    image,
    scalars='NIFTI',
    cmap='gray',
    opacity='sigmoid_9',
    show_scalar_bar=False,  
)

_ = pl.add_volume(
        image,
        scalars = "GTV__[GTV-1]",
        cmap=['white', 'cyan'],
        opacity='linear',
        opacity_unit_distance=np.mean(image.spacing),
        show_scalar_bar=False,
        shade = True
    )

_ = pl.add_volume(
        image,
        scalars = "GTV__[gtv-2]",
        cmap=['white', 'magenta'],
        opacity='linear',
        # opacity_unit_distance=np.mean(image.spacing),
        show_scalar_bar=False,
        shade = True
    )

_ = pl.add_volume(
        image,
        scalars = "GTV__[gtv-5]",
        cmap=['white', 'red'],
        opacity='linear',
        opacity_unit_distance=np.mean(image.spacing),
        show_scalar_bar=False,
        shade = True
    )

_ = pl.add_volume(
        image,
        scalars = "GTV__[gtv-6]",
        cmap=['white', 'lime'],
        opacity='linear',
        opacity_unit_distance=np.mean(image.spacing),
        show_scalar_bar=False,
        shade = True
    )

_ = pl.add_volume(
        image,
        scalars = "GTV__[gtv-7]",
        cmap=['white', 'yellow'],
        opacity='linear',
        opacity_unit_distance=np.mean(image.spacing),
        show_scalar_bar=False,
        shade = True
    )
pl.view_xz()
pl.camera.zoom(1.5)

pl.open_gif('../data/results/TCIA_NSCLC-Radiomics/visualization/volumes/LUNG1-294_shaded_gtv.gif')

nframe = 36
for phase in np.linspace(0, 360, nframe):
    pl.camera.azimuth = phase
    pl.write_frame()

pl.close()

Context leak detected, msgtracer returned -1
